In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/cs230/womens_edu

/content/drive/MyDrive/Colab Notebooks/cs230/womens_edu


In [3]:
import argparse
import logging
import os

import h5py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision import transforms, utils
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from skimage import io, transform
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

In [4]:
dataset_root_dir = '/content/drive/MyDrive/Colab Notebooks/cs230/womens_edu/data/'

In [ ]:
def split_train_val_test(df, train_yr, val_yr, test_yr):
  df['year'] = pd.to_numeric(df['year'])
  df = df.sample(frac=1, random_state=1234)

  train_df = df[ df['year'] <= train_yr  ]
  val_df = df[ (df['year'] > train_yr) & (df['year'] <= val_yr) ]
  test_df = df[ (df['year'] > val_yr) & (df['year'] <= test_yr) ]
  return train_df, val_df, test_df 

In [5]:
!git config --global user.email "disaaldan@gmail.com"
!git config --global user.name "disaalda"

In [ ]:
!git status

# Data Processing 

In [ ]:
df = pd.read_csv('data/filtered_sampled_ss.csv')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 0.1,DHSID_EA,year,cc,lat,lon,women_edu,path,img_captured_at,img_lon,img_lat,img_id,img_path
0,0,74101,74101,AM-2010-6#-00000175,2010,AM,40.865949,44.052637,10.307692,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1341142789364,44.118895,40.914849,455621522386245,AM/AM-2010-6#-00000175/455621522386245.jpeg
1,1,74102,74102,AM-2010-6#-00000176,2010,AM,40.878055,44.042707,10.761905,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1341142789364,44.118895,40.914849,455621522386245,AM/AM-2010-6#-00000176/455621522386245.jpeg
2,2,74103,74103,AM-2010-6#-00000215,2010,AM,40.776914,43.841243,12.476190,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1295470496500,43.853536,40.802320,448238129596253,AM/AM-2010-6#-00000215/448238129596253.jpeg
3,3,74104,74104,AM-2010-6#-00000218,2010,AM,40.808131,43.840526,12.600000,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1295470496500,43.852432,40.798289,803317710311828,AM/AM-2010-6#-00000218/803317710311828.jpeg
4,4,74105,74105,AM-2010-6#-00000232,2010,AM,40.765091,43.783366,11.480000,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1295470496500,43.853536,40.802320,448238129596253,AM/AM-2010-6#-00000232/448238129596253.jpeg


In [ ]:
df['img_path'] = dataset_root_dir + df['img_path']
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 0.1,DHSID_EA,year,cc,lat,lon,women_edu,path,img_captured_at,img_lon,img_lat,img_id,img_path
0,0,74101,74101,AM-2010-6#-00000175,2010,AM,40.865949,44.052637,10.307692,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1341142789364,44.118895,40.914849,455621522386245,/content/drive/MyDrive/Colab Notebooks/cs230/w...
1,1,74102,74102,AM-2010-6#-00000176,2010,AM,40.878055,44.042707,10.761905,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1341142789364,44.118895,40.914849,455621522386245,/content/drive/MyDrive/Colab Notebooks/cs230/w...
2,2,74103,74103,AM-2010-6#-00000215,2010,AM,40.776914,43.841243,12.476190,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1295470496500,43.853536,40.802320,448238129596253,/content/drive/MyDrive/Colab Notebooks/cs230/w...
3,3,74104,74104,AM-2010-6#-00000218,2010,AM,40.808131,43.840526,12.600000,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1295470496500,43.852432,40.798289,803317710311828,/content/drive/MyDrive/Colab Notebooks/cs230/w...
4,4,74105,74105,AM-2010-6#-00000232,2010,AM,40.765091,43.783366,11.480000,/content/drive/MyDrive/Colab Notebooks/cs230/w...,1295470496500,43.853536,40.802320,448238129596253,/content/drive/MyDrive/Colab Notebooks/cs230/w...


In [ ]:
# for now exclude these folders 
# df = df[ df['cc'] != 'BJ'] # need to resize
len(df)

7614

In [ ]:
# collapse the dataset s.t. each row is a satellite path with a list of img paths 
dhsid_df = df.groupby(['DHSID_EA', 'year', 'women_edu', 'path'])['img_path'].apply(list).reset_index()
dhsid_df.tail(10) 

,DHSID_EA,year,women_edu,path,img_path
1883,ZW-2015-7#-00000362,2015,12.030303,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1884,ZW-2015-7#-00000369,2015,10.391304,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1885,ZW-2015-7#-00000374,2015,10.406250,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1886,ZW-2015-7#-00000378,2015,10.741935,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1887,ZW-2015-7#-00000381,2015,11.303030,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1888,ZW-2015-7#-00000390,2015,11.521739,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1889,ZW-2015-7#-00000392,2015,9.800000,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1890,ZW-2015-7#-00000394,2015,9.594595,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1891,ZW-2015-7#-00000396,2015,9.750000,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...
1892,ZW-2015-7#-00000399,2015,11.243243,/content/drive/MyDrive/Colab Notebooks/cs230/w...,[/content/drive/MyDrive/Colab Notebooks/cs230/...


In [ ]:
dhsid_df['img_path'][100]

['/content/drive/MyDrive/Colab Notebooks/cs230/womens_edu/data/AM/AM-2016-7#-00000097/1161818567600875.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/cs230/womens_edu/data/AM/AM-2016-7#-00000097/3911978088838780.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/cs230/womens_edu/data/AM/AM-2016-7#-00000097/325791582223174.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/cs230/womens_edu/data/AM/AM-2016-7#-00000097/340195134193338.jpeg',
 '/content/drive/MyDrive/Colab Notebooks/cs230/womens_edu/data/AM/AM-2016-7#-00000097/316863043145729.jpeg']

In [ ]:
# dhsid_df.to_csv('data/multi_modal_input.csv')

# Dataset

In [ ]:
def get_street_tensor(img_path_list):
  image_tensor_list = [] 
  for img_path in img_path_list:
    image = io.imread(img_path)
    image = (image - image.min()) / (image.max() - image.min())
    image_tensor = torch.from_numpy(image)     
    image_tensor = image_tensor.permute(2,0,1).float()
    #print(image_tensor.shape)
    #print(img_path)
    image_tensor_list.append(image_tensor)

  return image_tensor_list

In [ ]:
def get_satellite_tensor(img_path):
    image = io.imread(img_path)
    image = image[:3]
    image = image[::-1]
    image = (image - image.min()) / (image.max() - image.min())
    image_tensor = torch.from_numpy(image)     

    return image_tensor 

In [ ]:
# Combined Dataset class 
class DHSIDDataset(Dataset):
  def __init__(self, df):
    self.satellite_path = df['path'].to_numpy()
    self.street_path = df['img_path'].to_numpy() # a list of paths 
    self.targets = df['women_edu'].to_numpy()

  def __len__(self):
    return self.satellite_path.shape[0] 

  def __getitem__(self, idx):
    sat_tensor = get_satellite_tensor(self.satellite_path[idx])
    street_tensor_list = get_street_tensor(self.street_path[idx]) # a list of tensors 
    target = torch.Tensor(np.array([self.targets[idx]]))

    return sat_tensor, street_tensor_list, target

# Model

In [ ]:
def create_model(model):
    if model == 'resnet18':
        model = models.resnet18(pretrained=True)
    elif model == 'resnet34':
        model = models.resnet34(pretrained=True)
    elif model == 'resnet50':
        model = models.resnet50(pretrained=True)
    model = nn.Sequential(*list(model.children())[:-1]) # get model only up to second to last layer 
    return model

In [ ]:
class SatelliteStreetModel(nn.Module):
  def __init__(self, sat_model, street_model):
    super(SatelliteStreetModel, self).__init__()

    self.satellite_model = create_model(sat_model)
    self.street_model = create_model(street_model)

    self.fc = nn.Linear(512*2, 1)

  def forward(self, sat_x, street_x):
    satellite_fv = self.satellite_model(sat_x)
    satellite_fv = torch.flatten(satellite_fv , start_dim=2)
    # print(satellite_fv.shape)
    street_fv = self.street_model(street_x) 
    street_fv = torch.mean(street_fv, 0)
    street_fv = torch.flatten(satellite_fv, start_dim=2)
    # print(satellite_fv.shape, street_fv.shape)
    # print(street_fv.shape)
    concat_fv = torch.cat( (satellite_fv, street_fv), dim=1)
    # print(concat_fv.shape)
    # TO DO: add more Dense/FC layers
    concat_fv = torch.squeeze(concat_fv)
    # print(concat_fv.shape)
    out = self.fc(concat_fv)
    # print(out.shape)
    return out

# Train

In [ ]:
# use GPU 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs, device, batch_size, verbose=True):
    batch_size = torch.from_numpy(np.array(batch_size)).to(device)
    
    for epoch in range(1, num_epochs + 1):
        batch_num = 1

        loss = 0.0
        optimizer.zero_grad()
        for sat_inputs, street_inputs, targets in train_loader:
            sat_inputs = sat_inputs.to(device)
            targets = targets.to(device)

            street_inputs = torch.stack(street_inputs, dim=1).squeeze(0)
            street_inputs = street_inputs.to(device)

            output = model(sat_inputs, street_inputs)
            # print(output, targets[0])
            loss += criterion(output, targets[0]) / batch_size

            if batch_num % batch_size.item() == 0:
              loss.backward()
              optimizer.step()
              optimizer.zero_grad()

              if verbose:                              
                print(f'Epoch [{epoch}/{num_epochs}], Step [{batch_num}/{len(train_loader)}], '
                      f'Loss: {loss.item():.4f}')
              
              loss = 0.0
            
            batch_num += 1

In [ ]:
BATCH_SIZE = 128
LEARNING_RATE = 0.003
NUM_EPOCHS = 50 # tune this 

In [ ]:
# TO TEST
dhsid_df = dhsid_df.sample(frac=1)
# dhsid_df = dhsid_df[:500,]

imgs = DHSIDDataset(dhsid_df)
loader = DataLoader(imgs, batch_size=1, num_workers=2)

model = SatelliteStreetModel('resnet18', 'resnet18') # resnet18 is the best model for satellite model
model = model.to(device)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-3)

In [ ]:
train_model(model, loader, criterion, optimizer, NUM_EPOCHS, device, BATCH_SIZE)

Epoch [1/50], Step [128/1893], Loss: 63.5259
Epoch [1/50], Step [256/1893], Loss: 56.7263
Epoch [1/50], Step [384/1893], Loss: 39.7190
Epoch [1/50], Step [512/1893], Loss: 31.1758
Epoch [1/50], Step [640/1893], Loss: 16.5413
Epoch [1/50], Step [768/1893], Loss: 10.7755
Epoch [1/50], Step [896/1893], Loss: 10.6921
Epoch [1/50], Step [1024/1893], Loss: 11.0203
Epoch [1/50], Step [1152/1893], Loss: 11.4650
Epoch [1/50], Step [1280/1893], Loss: 13.8155
Epoch [1/50], Step [1408/1893], Loss: 12.7738
Epoch [1/50], Step [1536/1893], Loss: 9.0178
Epoch [1/50], Step [1664/1893], Loss: 11.2462
Epoch [1/50], Step [1792/1893], Loss: 12.1072
Epoch [2/50], Step [128/1893], Loss: 11.0309
Epoch [2/50], Step [256/1893], Loss: 10.4906
Epoch [2/50], Step [384/1893], Loss: 9.9480
Epoch [2/50], Step [512/1893], Loss: 11.7824
Epoch [2/50], Step [640/1893], Loss: 9.4712
Epoch [2/50], Step [768/1893], Loss: 9.8867
Epoch [2/50], Step [896/1893], Loss: 10.4938
Epoch [2/50], Step [1024/1893], Loss: 11.1192
Epoch 

# Run

In [ ]:
# use GPU 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
# Hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 50
WEIGHT_DECAY = 1e-3

In [ ]:
print('Preparing the dataloader')
df = dhsid_df.sample(frac=1) # shuffle dataset 
train_df, val_df, test_df = split_train_val_test(df, 2017, 2017, 2018)

train_imgs = DHSIDDataset(train_df)
test_imgs = DHSIDDataset(test_df)
train_loader = DataLoader(train_imgs, batch_size=1, num_workers=2) # always set batch_size = 1 here 
test_loader = DataLoader(test_imgs, batch_size=1, num_workers=2)

model = SatelliteStreetModel('resnet18', 'resnet18') # resnet18 is the best model for satellite model 
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

Preparing the dataloader


In [ ]:
print(len(train_imgs), len(test_imgs))

1235 223


In [ ]:
train_model(model, train_loader, criterion, optimizer, NUM_EPOCHS, device, BATCH_SIZE)

RuntimeError: ignored

# Evaluate

In [ ]:
def evaluate(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    total = 0
    y_true = np.array([])
    y_pred = np.array([])
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            total += target.size(0)
            output = model(data)
            target = target.cpu().detach().numpy().squeeze()
            pred = output.cpu().detach().numpy().squeeze()
            y_true = np.append(y_true, target)
            y_pred = np.append(y_pred, pred)

  return y_true, y_pred

In [ ]:
y_test_preds, y_test = evaluate(test_loader, BATCH_SIZE, N_FEATURES)

In [ ]:
# print('Starting training')
# best_r2 = 0.
# r2 = -1

# for epoch in range(1, NUM_EPOCHS+1):
#   train(model, device, train_loader, optimizer, criterion, epoch)
#   r2, y_true, y_pred = test(model, device, test_loader, criterion, epoch)
#   if r2 >= best_r2:
#     best_r2 = r2
#     torch.save(model.state_dict(), SAVE_NAME + "/resnet18_model")
#     logging.info("\nSaved model with R2: {:.4f}\n".format(best_r2))
        
#     logging.info("\nBest R2: {:.4f}\n".format(best_r2))
#     print("\nBest R2: {:.4f}\n".format(best_r2))

In [ ]:
def calculate_metrics(pred, actual, verbose=True):
    result_metrics = {'mae' : mean_absolute_error(pred, actual),
                      'mape' : mean_absolute_percentage_error(pred, actual),
                      'mse' : mean_squared_error(pred, actual), 
                      'rmse' : mean_squared_error(pred, actual) ** 0.5
                      }
    
    if verbose:
      print("Mean Absolute Error:       ", result_metrics["mae"])
      print("Mean Absolute Percentage Error:       ", result_metrics["mape"])
      print("Mean Squared Error:   ", result_metrics["mse"])
      print("Root Mean Squared Error:   ", result_metrics["rmse"])
      
    return result_metrics

In [ ]:
metrics = calculate_metrics(y_pred, y_true)